<a href="https://colab.research.google.com/github/GORUP-3-AIR/-code/blob/main/%E3%80%8C%E7%A9%BA%E6%B1%99_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4 pandas fake-useragent



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.2 MB/s eta 0:00:00


In [ ]:
!pip install requests pandas

In [ ]:
pip install requests pandas tabulate


In [ ]:
import requests
import pandas as pd
from tabulate import tabulate

# ========== 1. 基本設定 ==========
API_KEY = "846e44e1-8cc5-4893-ad87-c79d2d383706"
BASE_URL = "https://data.moenv.gov.tw/api/v2/aqx_p_488"

YEAR = 2025
MONTH = 8
START_DAY = 3
END_DAY = 10
YEAR_MONTH = f"{YEAR}_{MONTH:02d}"

START_DT = f"{YEAR}-{MONTH:02d}-{START_DAY:02d} 00:00:00"
END_DT   = f"{YEAR}-{MONTH:02d}-{END_DAY:02d} 23:59:59"


# ========== 2. 爬資料函式（隱藏 offset 訊息） ==========
def fetch_aqi_history(api_key, year_month, start_dt, end_dt):
    all_records = []
    limit = 1000
    offset = 0
    filters = f"DataCreationDate,GR,{start_dt}|DataCreationDate,LE,{end_dt}"

    while True:
        params = {
            "format": "json",
            "offset": offset,
            "limit": limit,
            "api_key": api_key,
            "year_month": year_month,
            "filters": filters,
            "sort": "DataCreationDate asc"
        }

        resp = requests.get(BASE_URL, params=params, timeout=60)
        resp.raise_for_status()
        data = resp.json()
        records = data.get("records", [])

        if not records:
            break

        all_records.extend(records)
        if len(records) < limit:
            break

        offset += limit

    return all_records


# ========== 3. 抓資料 + 整理欄位 ==========
records = fetch_aqi_history(API_KEY, YEAR_MONTH, START_DT, END_DT)
df = pd.DataFrame(records)

if df.empty:
    print("⚠️ 沒有抓到資料，請確認日期或月份設定。")
else:
    df["datacreationdate"] = pd.to_datetime(df["datacreationdate"])
    df["西元年"] = df["datacreationdate"].dt.year
    df["幾月幾號"] = df["datacreationdate"].dt.strftime("%m/%d")
    df["資料生成時間"] = df["datacreationdate"].dt.strftime("%Y-%m-%d %H:%M:%S")

    df_simple = df[[
        "西元年", "幾月幾號", "sitename", "county",
        "pollutant", "status", "資料生成時間"
    ]].rename(columns={
        "sitename": "測站名稱",
        "county": "所屬縣市",
        "pollutant": "主要汙染物",
        "status": "狀態描述"
    })

    # ========== 4. 表格輸出（帶框線） ==========
    print(tabulate(df_simple.head(10), headers="keys", tablefmt="grid", showindex=False))
    print(f"\n✅ 共 {len(df_simple)} 筆資料")

    # ========== 5. 存成 CSV ==========
    output_file = f"aqi_simple_{YEAR}{MONTH:02d}_{START_DAY:02d}_{END_DAY:02d}.csv"
    df_simple.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"💾 已輸出：{output_file}")


+----------+------------+------------------+------------+--------------+------------+---------------------+
|   西元年 | 幾月幾號   | 測站名稱         | 所屬縣市   | 主要汙染物   | 狀態描述   | 資料生成時間        |
+==========+============+==================+============+==============+============+=====================+
|     2025 | 08/03      | 屏東(枋山)       | 屏東縣     |              |            | 2025-08-03 01:00:00 |
+----------+------------+------------------+------------+--------------+------------+---------------------+
|     2025 | 08/03      | 臺南（南化）     | 臺南市     |              | 良好       | 2025-08-03 01:00:00 |
+----------+------------+------------------+------------+--------------+------------+---------------------+
|     2025 | 08/03      | 新北(樹林)       | 新北市     |              | 良好       | 2025-08-03 01:00:00 |
+----------+------------+------------------+------------+--------------+------------+---------------------+
|     2025 | 08/03      | 台中市（和平區） | 臺中市     |              | 良好       | 2025-08-03 0

1.執行上 需要下載CSV檔上傳檔案 才能讓程式碼去抓出來合併 還在想方法可以可以直接網上抓
2.

In [ ]:
import pandas as pd
import requests

# =============================
# 📘 讀取資料
# =============================
df_air = pd.read_csv("空氣品質監測月值.csv")     # 空氣品質多年月資料
df_pneu = pd.read_csv("NHI_OtherPneumonia.csv")  # 肺炎資料
API_KEY = "e75b1660-e564-4107-aad5-a8be1f905dd9"

# =============================
# 🗓️ 自動偵測年月範圍
# =============================
if "monitormonth" not in df_air.columns:
    raise ValueError("⚠️ 找不到欄位 'monitormonth'，請確認檔案格式。")

df_air["西元年"] = df_air["monitormonth"] // 100
df_air["月份"] = df_air["monitormonth"] % 100

year_min = int(df_air["西元年"].min())
year_max = int(df_air["西元年"].max())
month_min = int(df_air["月份"].min())
month_max = int(df_air["月份"].max())

print(f"📅 資料涵蓋期間：{year_min}年{month_min:02d}月 ～ {year_max}年{month_max:02d}月")

# =============================
# 🔹 互動輸入年月
# =============================
target_year = int(input(f"請輸入查詢年份（例如 {year_max}）："))
target_month = int(input("請輸入查詢月份（1-12）："))

# =============================
# 🌫️ 篩選指定年月
# =============================
air_month = df_air[
    (df_air["西元年"] == target_year) & (df_air["月份"] == target_month)
].copy()

if air_month.empty:
    print(f"⚠️ 找不到 {target_year} 年 {target_month} 月的空氣品質資料，請確認 CSV 是否有該月份。")
else:
    air_month = air_month[
        air_month["itemengname"].astype(str).str.upper() == "PM2.5"
    ].copy()

    air_month["空氣品質"] = pd.to_numeric(air_month["concentration"], errors="coerce")
    air_month["日期"] = pd.to_datetime(
        air_month["monitormonth"].astype(str) + "01", format="%Y%m%d", errors="coerce"
    )
    air_month["月"] = air_month["日期"].dt.month
    air_month["日"] = air_month["日期"].dt.day
    air_month = air_month.rename(columns={"sitename": "監測地區"})

    # =============================
    # 🗺️ 從 API 抓監測站的縣市對應
    # =============================
    url_site = "https://data.moenv.gov.tw/api/v2/aqx_p_35"
    resp_site = requests.get(url_site, params={"api_key": API_KEY, "format": "json", "limit": 1000})
    df_site = pd.DataFrame(resp_site.json()["records"])
    df_site = df_site[["sitename", "county"]].drop_duplicates()
    df_site.columns = ["監測地區", "地區"]
    df_site["地區"] = df_site["地區"].str.replace("臺", "台", regex=False)

    air_month = air_month.merge(df_site, on="監測地區", how="left")

    # =============================
    # 🩺 處理肺炎資料（以週轉月）
    # =============================
    df_pneu["年"] = pd.to_numeric(df_pneu["年"], errors="coerce")
    df_pneu["週"] = pd.to_numeric(df_pneu["週"], errors="coerce")
    df_pneu["其他肺炎健保就診人次"] = pd.to_numeric(df_pneu["其他肺炎健保就診人次"], errors="coerce")

    pneu = df_pneu.dropna(subset=["年", "週", "縣市"]).copy()
    pneu["週起始日"] = pd.to_datetime(
        pneu["年"].astype(str) + pneu["週"].astype(str) + "1",
        format="%G%V%u", errors="coerce"
    )
    pneu["月份"] = pneu["週起始日"].dt.month

    pneu_city = (
        pneu[(pneu["年"] == target_year) & (pneu["月份"] == target_month)]
        .groupby(["縣市"], as_index=False)["其他肺炎健保就診人次"]
        .sum()
        .rename(columns={"縣市": "地區", "其他肺炎健保就診人次": "肺炎就診人數"})
    )
    pneu_city["地區"] = pneu_city["地區"].str.replace("臺", "台", regex=False)

    # =============================
    # 🔗 合併資料
    # =============================
    final = air_month.merge(pneu_city, on="地區", how="left")

    # 🌀 新增空氣狀態分類
    def classify_air(pm25):
        if pd.isna(pm25):
            return "無資料"
        elif pm25 <= 15:
            return "良好"
        elif pm25 <= 35:
            return "普通"
        elif pm25 <= 54:
            return "對敏感族群不健康"
        elif pm25 <= 70:
            return "對所有族群不健康"
        else:
            return "非常不健康 / 危險"

    final["空氣狀態"] = final["空氣品質"].apply(classify_air)

    # 整理欄位順序
    final = final[[
        "西元年", "月", "日", "地區", "監測地區", "空氣品質", "空氣狀態", "肺炎就診人數"
    ]]

    # =============================
    # 💾 輸出結果
    # =============================
    print(f"\n🎯 {target_year} 年 {target_month} 月 各縣市空氣品質與肺炎就診人數（前20筆）：")
    display(final.head(20))

    output_name = f"{target_year}年{target_month}月_各縣市空氣品質_肺炎.csv"
    final.to_csv(output_name, index=False, encoding="utf-8-sig")
    print(f"✅ 已輸出：{output_name}")


📅 資料涵蓋期間：2001年01月 ～ 2025年12月
請輸入查詢年份（例如 2025）：2024
請輸入查詢月份（1-12）：12

🎯 2024 年 12 月 各縣市空氣品質與肺炎就診人數（前20筆）：


,西元年,月,日,地區,監測地區,空氣品質,空氣狀態,肺炎就診人數
0,2024,12,1,基隆市,基隆,10.0,良好,3917.0
1,2024,12,1,新北市,汐止,13.0,良好,29533.0
2,2024,12,1,NaN,萬里,12.0,良好,NaN
3,2024,12,1,新北市,新店,11.0,良好,29533.0
4,2024,12,1,新北市,土城,11.0,良好,29533.0
5,2024,12,1,新北市,板橋,11.0,良好,29533.0
6,2024,12,1,新北市,新莊,15.0,良好,29533.0
7,2024,12,1,新北市,菜寮,13.0,良好,29533.0
8,2024,12,1,新北市,林口,11.0,良好,29533.0
9,2024,12,1,新北市,淡水,12.0,良好,29533.0


✅ 已輸出：2024年12月_各縣市空氣品質_肺炎.csv
